# Tutorial

This tutorial provides a deep dive into the **Online Retail Simulator**, walking through the three-phase data generation process, exploring the generated data, and demonstrating how to inject and measure treatment effects using the enrichment layer.

The simulator operates in three phases. First, the **products** phase generates a product catalog with key attributes such as category and price tier. Second, the **product_details** phase enriches each product with detailed content including brand names, titles, descriptions, and feature lists. Third, the **metrics** phase simulates sales transactions with realistic temporal patterns and conversion metrics. You can choose between **rule-based generation**, which is deterministic and interpretable, and **synthesizer-based generation**, which learns patterns from your own data. An **enrichment layer** then allows you to inject known treatment effects—such as a quantity boost or a gradual rollout—so you can validate whether your causal models recover the true effect size.

## Simulate Product Data

We start by importing the **Online Retail Simulator** alongside a few standard libraries.

In [ ]:
# Standard Library
import inspect

# Third-party packages
from IPython.display import Code
import pandas as pd

from online_retail_simulator import enrich, simulate, load_job_results

# Local imports
from support import (
    plot_revenue_by_category,
    plot_daily_metrics_trend,
    plot_conversion_funnel,
    plot_treatment_effect,
    display_product_details,
)

Let's start by simulating 100 products. The configuration is defined using [YAML](https://yaml.org/) and organizes the three-phase simulation process.

The **STORAGE** section specifies where to save output files.

The **RULE** section contains two subsections—**PRODUCTS** and **METRICS**—each defining a `FUNCTION` to call and a `PARAMS` subsection for function-specific parameters like `num_products`, `date_start`, `date_end`, and `seed`.

The **PRODUCT_DETAILS** section specifies the `FUNCTION` used to enrich products with additional content.

In [ ]:
! cat "config_tutorial.yaml"

To understand how the simulation functions work under the hood, you can explore their source code directly. Python's `inspect` module lets you view the implementation right inside the notebook, or you can browse the source files on GitHub ([products](https://github.com/eisenhauerIO/tools-online-retail-simulator/blob/main/online_retail_simulator/simulate/products_rule_based.py), [product_details](https://github.com/eisenhauerIO/tools-online-retail-simulator/blob/main/online_retail_simulator/simulate/product_details_mock.py), [metrics](https://github.com/eisenhauerIO/tools-online-retail-simulator/blob/main/online_retail_simulator/simulate/metrics_rule_based.py)) or the API documentation ([products](https://eisenhauerio.github.io/tools-online-retail-simulator/_modules/online_retail_simulator/simulate/products_rule_based.html#simulate_products_rule_based), [product_details](https://eisenhauerio.github.io/tools-online-retail-simulator/_modules/online_retail_simulator/simulate/product_details.html#simulate_product_details_mock), [metrics](https://eisenhauerio.github.io/tools-online-retail-simulator/_modules/online_retail_simulator/simulate/metrics_rule_based.html#simulate_metrics_rule_based)).

In [ ]:
from online_retail_simulator.simulate.products_rule_based import simulate_products_rule_based

Code(inspect.getsource(simulate_products_rule_based), language="python")

In [ ]:
from online_retail_simulator.simulate.product_details_mock import simulate_product_details_mock

Code(inspect.getsource(simulate_product_details_mock), language="python")

Calling `simulate("config_tutorial.yaml")` triggers a three-phase data generation process. First, it reads and validates the YAML configuration, then runs the **products** phase to create a product catalog with the specified number of products, assigning each a unique identifier, category, and price. Second, it runs the **product_details** phase to enrich each product with brand names, titles, descriptions, and feature lists. Third, it runs the **metrics** phase, simulating a shopper conversion funnel (impressions, visits, cart additions, orders, and revenue) for each product across the defined date range. The function writes the generated `DataFrame`s to disk and returns a `JobInfo` object that tracks where the results are stored—which you then pass to `load_job_results()` to retrieve the data.

The products data represents the catalog of items available for sale. Each product starts with core **attributes**—a unique `product_identifier`, `category` (such as Electronics, Clothing, or Books), `brand`, and `price`—which are then enriched with **product_details** including `title`, `description`, and `features`. These attributes influence the simulated sales behavior—for example, higher-priced items may have different conversion patterns than low-cost products, and categories can exhibit distinct purchasing dynamics.

In [ ]:
job_info = simulate("config_tutorial.yaml")

results = load_job_results(job_info)
products = results["products"]
metrics = results["metrics"]

### Products

In [ ]:
products.head()

Let's examine a single product to understand the structure better.

In [ ]:
example_product = products.iloc[0]
example_product

This product has a unique `product_identifier` for tracking, belongs to the `category` Clothing with a `price` of $19.63, and includes enriched content: a `brand` name (StyleFit), a descriptive `title`, a marketing-focused `description`, and a list of product `features`. These attributes combine catalog characteristics with content details to create a realistic product listing.

### Metrics

In [ ]:
metrics.sort_values(["product_identifier", "date"]).head()

Let's examine a single metric record to understand the conversion funnel structure.

In [ ]:
example_metric = metrics.iloc[0]
example_metric

This metric record tracks the shopper journey as a conversion funnel for one product on one date. It includes `product_identifier`, `category`, and `date` for identification, then follows shoppers through each stage: `impressions` (product shown), `visits` (clicked to view), `cart_adds` (added to cart), `ordered_units` (purchased), and `revenue` (total sales). This funnel structure captures how shoppers drop off at each stage of the purchase process.

### Exploration

Now that we understand the structure of both DataFrames, let's explore the generated data through visualizations and summary statistics.

<div class="alert alert-block alert-info">
<b>Things to explore</b>
<ul>
    <li>What is the date range?</li>
    <li>How many categories?</li>
    <li>What is the total revenue?</li>
</ul>
</div>

How is revenue distributed across product categories?

In [ ]:
category_revenue = metrics.groupby("category")["revenue"].sum().sort_values()
plot_revenue_by_category(category_revenue)

In [ ]:
Code(inspect.getsource(plot_revenue_by_category), language="python")

How do metrics vary over time?

In [ ]:
daily_metrics = metrics.groupby("date").agg({"ordered_units": "sum", "revenue": "sum"}).reset_index()
plot_daily_metrics_trend(daily_metrics)

How do shoppers move through the purchase journey?

In [ ]:
plot_conversion_funnel(metrics)

<div class="alert alert-block alert-info">
<b>Things to explore</b>
<ul>
    <li>What fraction of impressions become visits?</li>
    <li>What fraction of visits add to cart?</li>
    <li>What fraction of cart adds become orders?</li>
    <li>What is the overall conversion rate?</li>
</ul>
</div>

## Enrich Product Data

Real-world e-commerce teams constantly face a critical question: **Does improving product content quality increase sales?** To help you explore this with known ground truth, the simulator lets you inject treatment effects into a subset of products starting at a specific date, creating synthetic experiments where you know the true causal impact.

The enrichment process operates through two coordinated mechanisms:

1. **Content Enhancement**: Generate upgraded product copy with premium brand positioning, compelling descriptions, and detailed feature highlights
2. **Sales Impact**: Apply the corresponding sales boost that this improved content would realistically produce

The **ENRICHMENT** configuration controls the treatment parameters. The `effect_size` controls the magnitude of the boost (0.5 = 50% increase), `enrichment_fraction` determines what share of products receive treatment (1.0 = 100%), and `enrichment_start` sets when the intervention begins. The `ramp_days` parameter models a gradual rollout where the effect linearly increases over the specified number of days, mimicking real-world deployment patterns.

In [ ]:
! cat "config_tutorial_enrichment.yaml"

In [ ]:
from online_retail_simulator.enrich.enrichment_library import product_detail_boost

Code(inspect.getsource(product_detail_boost), language="python")

Calling `enrich("config_tutorial_enrichment.yaml", job_info)` applies the treatment effect to the simulated data. The `product_detail_boost()` function performs two operations: first, it regenerates product details for the treatment group using `treatment_mode=True` (creating enhanced content with premium brand names, compelling descriptions, and specific feature highlights); second, it applies a 50% sales boost to these products starting November 15th. The function automatically saves three artifacts: the original product details, the enhanced treatment product details, and the enriched sales data—which we can load and examine to understand both the content improvements and their impact on sales metrics.

In [ ]:
enriched_job = enrich("config_tutorial_enrichment.yaml", job_info)
results = load_job_results(enriched_job)

### Changes

In [ ]:
enriched = results["enriched"]
enriched.head()

The enriched dataset contains the same metrics as before (`impressions`, `visits`, `cart_adds`, `ordered_units`, `revenue`) but now includes an `enriched` column indicating which products received the treatment. Products with `enriched=True` have enhanced content and boosted sales metrics starting from the treatment date.

In [ ]:
example_enriched = enriched.iloc[0]
example_enriched

This record shows all the sales funnel metrics plus the `enriched` indicator. The treatment assignment is random but deterministic (controlled by the `seed` parameter), allowing us to create a reproducible experiment where we know the ground truth effect size.

The enrichment process modifies both sales metrics and product content. Let's compare the original and enhanced product details to see the transformation.

In [ ]:
control_products = results["product_details_original"]
control_example = control_products.iloc[0]

display_product_details(control_example, "CONTROL GROUP (Original Content)")

In [ ]:
treatment_products = results["product_details_enriched"]
treatment_example = treatment_products.iloc[0]

display_product_details(treatment_example, "TREATMENT GROUP (Enhanced Content)", add_newline=True)

<div class="alert alert-block alert-info">
<b>Things to explore</b>
<ul>
    <li>What differences do you notice in brand names?</li>
    <li>How do the titles differ between control and treatment?</li>
    <li>What makes the treatment description more compelling?</li>
    <li>How are the features enhanced in the treatment version?</li>
</ul>
</div>

### Impact

The treatment effect visualization compares daily revenue between the original and enriched datasets. Before the treatment start date, the lines should overlap perfectly. After November 15th, the enriched dataset should show higher revenue due to the 50% boost applied to all products.

In [ ]:
plot_treatment_effect(metrics, enriched, "2024-11-15")